In [96]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [97]:
desc_infile = '..\\dataset\\description_0.docx'
# req_infile = '..\\dataset\\requirements_0_reduced_6.docx'
req_infile = '..\\dataset\\requirements_0_reduced_6_part.docx'

In [98]:
from attr import dataclass
import docx


@dataclass
class HeadingInfo:
    text: str
    inheritance_number: int

class HeadingsTracker:
    def __init__(self):
        self.headings_stack = []
        pass

    def next_elem(self, elem) -> bool:
        HEADING_STYLE_NAME = "Heading"
        if isinstance(elem, docx.text.paragraph.Paragraph):
            style_name = elem.style.name
            if style_name.startswith(HEADING_STYLE_NAME):
                inheritance_number = int(style_name[len(HEADING_STYLE_NAME):])

                # remove lover headings from stack
                while(len(self.headings_stack) > 0 and self.headings_stack[-1].inheritance_number >= inheritance_number):
                    del self.headings_stack[-1]

                if len(elem.text) > 0: 
                    self.headings_stack.append(HeadingInfo(elem.text, inheritance_number))
                return True
        return False

    def get_text(self):
        s = ""
        for info in self.headings_stack:
            s += '#'*info.inheritance_number + ' ' + info.text + '\n'
        return s

@dataclass
class TextPartInfo:
    heading: str
    body: str
    id: int
    # TODO:
    # chapter_number: int
    
def parse_docx_parts(file_path: str) -> list[TextPartInfo]:  
    document = docx.Document(file_path)
    htracker = HeadingsTracker()

    # for paragraph in document.paragraphs:
    text_parts = []
    cur_heading = ''
    cur_body = ''
    prev_is_normal = False
    for paragraph in document.iter_inner_content():
        cur_is_normal = False
        if isinstance(paragraph, docx.text.paragraph.Paragraph):
            if not htracker.next_elem(paragraph):
                if not prev_is_normal or len(paragraph.text) > 200:
                    if len(cur_body) > 0:
                        part = TextPartInfo(cur_heading, cur_body, len(text_parts))
                        text_parts.append(part)
                    cur_heading = htracker.get_text()
                    cur_body = ''
                if len(paragraph.text) > 0 and not paragraph.text.isspace():
                    cur_body += paragraph.text + '\n'
                    cur_is_normal = True
        elif isinstance(paragraph, docx.table.Table):
            for row in paragraph.rows:
                for cell in row.cells:
                    cur_body += f"| {cell.text} "
                cur_body += "|\n"
        prev_is_normal = cur_is_normal
    
    return text_parts


## Step 1: Parsing

In [99]:
req_text_parts = parse_docx_parts(req_infile)
desc_text_parts = parse_docx_parts(desc_infile)

In [100]:
print(f"{len(req_text_parts)=}")
print(f"{len(desc_text_parts)=}")

len(req_text_parts)=71
len(desc_text_parts)=70


In [101]:
# FIXME: for tests
# req_text_parts = req_text_parts[:10]
# desc_text_parts = desc_text_parts[10:20]

## Step 2: technical requirement points finding

In [102]:
import request as req
import api_ollama
import api_yandex

llm_embed = api_yandex.YandexAPIRequest(model_name="yandexgpt")
# llm_api = api_ollama.LlamaAPIRequest(model_name="llama3")
llm_api = api_yandex.YandexAPIRequest(model_name="yandexgpt")

In [104]:
from enum import Enum
import numpy as np
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

class RequirementStatus(Enum):
    NO_INFORMATION = 0
    NOT_SATISFIED = 1
    PARTLY_SATISFIED = 2
    SATISFIED = 3

@dataclass
class RequirementPointInfo:
    text: str
    req_part_id: int
    embedding: np.array
    desc_part_id: int = -1
    status: RequirementStatus = RequirementStatus.NO_INFORMATION
    comment: str = ''
    # TODO:
    # req_chapter_id: int
    # desc_chapter_id: int

async_res_s = []
for req_part_id, text_req in enumerate(req_text_parts):
    text = text_req.heading + text_req.body
    async_res_s.append(req.request(llm_api, text=text, role=0)[0])

INFO:root:Yandex:yandexgpt request: {
    "modelUri": "gpt://b1gl3l0kiqfpqo9lhsgs/yandexgpt/latest",
    "completionOptions": {
        "stream": false,
        "temperatute": 0,
        "maxTokens": 2000
    },
    "messages": [
        {
            "role": "system",
            "text": "Ты профессиональный опытный инженер. Твоя задача найти в тексте ниже строгие технические требования компании-закупщика к оборудованию. Если в тексте нет технических требований, напиши `Требований нет`. Ответом должен быть список. Ответ не должен содержать юридическую информацию. Ответ должен быть максимально лаконичным и содержать только техническую информацию. Ответ должен быть на русском языке."
        },
        {
            "role": "user",
            "text": "# Общие требования\nНасосное оборудование (включая вспомогательные системы), на которое распространяется настоящий стандарт, должно конструироваться и изготавливаться в расчете на срок службы не менее 20 лет (за исключением естественно из

In [105]:
points = []
for req_part_id, async_res in enumerate(async_res_s):
    cur_points = req.extract_points(async_res.get())
    
    for p in cur_points:
        # create embedding
        emb = llm_embed.get_embedding(p)
        # save point text with embedding
        points.append(RequirementPointInfo(p, req_part_id, emb))
    
    logging.info(f"Detected points: {cur_points}")

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): llm.api.cloud.yandex.net:443
DEBUG:urllib3.connectionpool:https://llm.api.cloud.yandex.net:443 "GET /operations/d7qf20ng3qhob9nju54n HTTP/1.1" 200 1933
DEBUG:root:Yandex: get async response: {"id":"d7qf20ng3qhob9nju54n","description":"Async GPT Completion","createdAt":"2024-04-24T22:47:28Z","createdBy":"aje6aon7qihc9991fqk8","modifiedAt":"2024-04-24T22:47:33Z","done":true,"metadata":null,"response":{"@type":"type.googleapis.com/yandex.cloud.ai.foundation_models.v1.CompletionResponse","alternatives":[{"message":{"role":"assistant","text":"1. Срок службы насосного оборудования — не менее 20 лет (за исключением естественно изнашиваемых деталей).\n\n2. Срок непрерывной эксплуатации — не менее 3 лет.\n\n3. Остановка оборудования для выполнения техобслуживания или проверки не является нарушением требования к сроку службы.\n\n4. Поставщик берёт на себя ответственность за всё оборудование и все вспомогательные системы, включённые 

In [110]:
print(f"{len(points)=}")

len(points)=176


In [106]:
for p in points:
    print(f"{p.req_part_id=}, {p.text=}")

p.req_part_id=0, p.text='Срок службы насосного оборудования — не менее 20 лет (за исключением естественно изнашиваемых деталей).'
p.req_part_id=0, p.text='Срок непрерывной эксплуатации — не менее 3 лет.'
p.req_part_id=0, p.text='Остановка оборудования для выполнения техобслуживания или проверки не является нарушением требования к сроку службы.'
p.req_part_id=0, p.text='Поставщик берёт на себя ответственность за всё оборудование и все вспомогательные системы, включённые в объём заказа.'
p.req_part_id=0, p.text='Заказчик и поставщик несут ответственность за передачу достоверной информации друг другу.'
p.req_part_id=0, p.text='Заказчик должен указать условия эксплуатации, условия на площадке и технические условия, включая все данные, указанные в рекомендуемой форме опросных листов.'
p.req_part_id=0, p.text='Заказчик должен предоставить поставщику паспорт перекачиваемой среды при изменении параметров перекачиваемой среды.'
p.req_part_id=1, p.text='NPSHA;'
p.req_part_id=1, p.text='кривая те

## Step 3: match requirements with description  

In [107]:
from vector_database import VectorDatabase

# get embeddings for description parts
desc_embeddings_db = VectorDatabase()
for desc in desc_text_parts:
    emb = llm_embed.get_embedding(desc.heading + desc.body)
    desc_embeddings_db.add_embedding(emb, desc)

INFO:root:Yandex:yandexgpt request embed: {'modelUri': 'emb://b1gl3l0kiqfpqo9lhsgs/text-search-query/latest', 'text': 'СОДЕРЖАНИЕ\n'}
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): llm.api.cloud.yandex.net:443
DEBUG:urllib3.connectionpool:https://llm.api.cloud.yandex.net:443 "POST /foundationModels/v1/textEmbedding HTTP/1.1" 200 None
INFO:root:Yandex:yandexgpt request embed: {'modelUri': 'emb://b1gl3l0kiqfpqo9lhsgs/text-search-query/latest', 'text': 'Настоящие технические условия распространяются на агрегаты электронасосные торговой марки Villina вертикальные герметичные типа "Villina-ГНВЦ" (далее агрегаты), предназначенные для перекачивания в стационарных условиях нейтральных, агрессивных, токсичных, а также взрыво- и пожароопасных жидкостей, пары которых могут образовывать с воздухом взрывоопасные смеси категорий IIA, IIB, IIC, групп T1, Т2, Т3, Т4 по ГОСТ Р МЭК 60079-20-1.\nАгрегаты не предназначены для перекачивания кристаллизующихся и полимеризующихся жидкостей.\t

In [108]:
desc_embeddings_db.build_index()

In [111]:
import tqdm

async_responses = []
for point_id in tqdm.tqdm(range(len(points))):
    cur_point = points[point_id]
    
    # search k most common parts in description
    # TODO: adjust K
    K = 2
    kn_s = desc_embeddings_db.search(cur_point.embedding)
    
    cur_async_responses = []
    for neighbor in kn_s:
        desc = neighbor[0]
        desc_text = desc.heading + desc.body
        res = req.request(llm_api, text=desc_text, clauses=[cur_point.text], role=2)
        cur_async_responses.append(res)
    async_responses.append(cur_async_responses)

  0%|          | 0/176 [00:00<?, ?it/s]INFO:root:Yandex:yandexgpt request: {
    "modelUri": "gpt://b1gl3l0kiqfpqo9lhsgs/yandexgpt/latest",
    "completionOptions": {
        "stream": false,
        "temperatute": 0,
        "maxTokens": 2000
    },
    "messages": [
        {
            "role": "system",
            "text": "Ты профессиональный опытный инженер. Ты на вход будешь получать текст с техническим описанием. Определи удовлетворяет ли текст пункту технического требования. Ответами могут быть: нет информации (о пункте ничего не сказано в тексте), несоответствует (имеется противоречивая информация), частично соответствует (нехватает информации), соответствует (информация в пункте полностью соответствует информации текста). Ответ должен быть на русском языке. Пункт технического требования: Срок службы насосного оборудования — не менее 20 лет (за исключением естественно изнашиваемых деталей)."
        },
        {
            "role": "user",
            "text": "# Технические т

In [ ]:
def classify_response(message) -> RequirementStatus:
    lower_message = message.lower()
    if "нет информации" in lower_message:
        reply = RequirementStatus.NO_INFORMATION
    elif lower_message.find("не соотвеству") != -1 or "противоречивая информация" in lower_message:
        reply = RequirementStatus.NOT_SATISFIED
    elif lower_message.find("частично соответству") != -1:
        reply = RequirementStatus.PARTLY_SATISFIED
    elif lower_message.find("соответству") != -1:
        reply = RequirementStatus.SATISFIED
    else:
        logging.warn(f"Message without clear answer: {message}")
        reply = RequirementStatus.NO_INFORMATION
    return reply

In [112]:
for point_id, async_res_s in zip(range(len(points)), async_responses):
    cur_point = points[point_id]
    
    for neighbor_res in async_res_s:
        response = neighbor_res[0].get()
        cur_status = classify_response(response)
        if cur_status.value > cur_point.status.value:
            cur_point.status = cur_status
            cur_point.desc_part_id = desc.id
            cur_point.comment = response
        if cur_status == RequirementStatus.SATISFIED:
            break
        
    points[point_id] = cur_point

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): llm.api.cloud.yandex.net:443
DEBUG:urllib3.connectionpool:https://llm.api.cloud.yandex.net:443 "GET /operations/d7q8gqmnndhg5s93h69u HTTP/1.1" 200 700
DEBUG:root:Yandex: get async response: {"id":"d7q8gqmnndhg5s93h69u","description":"Async GPT Completion","createdAt":"2024-04-24T22:53:26Z","createdBy":"aje6aon7qihc9991fqk8","modifiedAt":"2024-04-24T22:53:28Z","done":true,"metadata":null,"response":{"@type":"type.googleapis.com/yandex.cloud.ai.foundation_models.v1.CompletionResponse","alternatives":[{"message":{"role":"assistant","text":"**Соответствует.**\n\nВ тексте есть вся необходимая информация о ресурсе службы насоса и сроке службы агрегата."},"status":"ALTERNATIVE_STATUS_FINAL"}],"usage":{"inputTextTokens":"211","completionTokens":"21","totalTokens":"232"},"modelVersion":"07.03.2024"}}
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): llm.api.cloud.yandex.net:443
DEBUG:urllib3.connectionpool:https://llm

In [113]:
for p in points:
    print(f"{p.status}: {p.text=}")

RequirementStatus.SATISFIED: p.text='Срок службы насосного оборудования — не менее 20 лет (за исключением естественно изнашиваемых деталей).'
RequirementStatus.PARTLY_SATISFIED: p.text='Срок непрерывной эксплуатации — не менее 3 лет.'
RequirementStatus.NO_INFORMATION: p.text='Остановка оборудования для выполнения техобслуживания или проверки не является нарушением требования к сроку службы.'
RequirementStatus.NO_INFORMATION: p.text='Поставщик берёт на себя ответственность за всё оборудование и все вспомогательные системы, включённые в объём заказа.'
RequirementStatus.NO_INFORMATION: p.text='Заказчик и поставщик несут ответственность за передачу достоверной информации друг другу.'
RequirementStatus.PARTLY_SATISFIED: p.text='Заказчик должен указать условия эксплуатации, условия на площадке и технические условия, включая все данные, указанные в рекомендуемой форме опросных листов.'
RequirementStatus.NO_INFORMATION: p.text='Заказчик должен предоставить поставщику паспорт перекачиваемой сре

In [117]:
import csv

out_csv_file = "./result.csv"
headers = ['Пункт СТО ИНТИ', 'Требование', 'Пункт ТУ/СТО', 'Формулировка ТУ/СТО', 'Статус (С/Ч/Н)', 'Комментарий']
with open(out_csv_file, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    # Write headers
    writer.writerow(headers)
    # Write data rows
    for p in points:
        status = ('С' if p.status == RequirementStatus.SATISFIED 
                  else 'Ч' if p.status == RequirementStatus.PARTLY_SATISFIED else 'Н')
        writer.writerow([p.req_part_id, p.text, p.desc_part_id, '', status, p.comment])